# EE 461P: Sales Prediction

In [10]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from itertools import product

from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from xgboost import XGBRegressor, plot_importance
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from itertools import product
import gc
import re
from sklearn.model_selection import GridSearchCV

In [11]:
# make a ton of dataframes for all files
df_items = pd.read_csv('items.csv')
df_cat = pd.read_csv('item_categories.csv')
df_sales = pd.read_csv('sales_train.csv')
df_test = pd.read_csv('test.csv')
df_shops = pd.read_csv('shops.csv')
sample = pd.read_csv('sample_submission.csv')

In [3]:
# look at the shops
# df_shops['shop_name'].value_counts()

### Preprocessing

In [12]:
# fix shops that are the same thing
df_shops.loc[df_shops['shop_id']==0, 'shop_id'] = 57
df_shops.loc[df_shops['shop_id']==1, 'shop_id'] = 58
df_shops.loc[df_shops['shop_id']==10, 'shop_id'] = 11
# df_shops

In [13]:
df_dropped = df_sales.drop(['date','item_price'], axis=1)

df_group = df_dropped.groupby(['date_block_num', 'shop_id', 'item_id']).sum().reset_index()
df_train = df_group.pivot_table(index=['shop_id','item_id'], columns='date_block_num', values='item_cnt_day', 
                        fill_value=0)

df_train.reset_index(inplace=True)
df_train.head()

date_block_num,shop_id,item_id,0,1,2,3,4,5,6,7,...,24,25,26,27,28,29,30,31,32,33
0,0,30,0,31,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,31,0,11,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,32,6,10,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,33,3,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,35,1,14,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
df_forecast_dropped = df_test.drop(['ID'], axis=1)
df_test_actual = pd.merge(df_forecast_dropped, df_train, on=['shop_id','item_id'], how='left').fillna(0)

### Model writing - DTR, RFR No Tune

In [6]:
X = df_train[df_train.columns[:-1]].values
y = df_train[df_train.columns[-1]]
print(X.shape)
print(y.shape)

(424124, 35)
(424124,)


In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, test_size=0.15, random_state=15)

In [8]:
dtr = DecisionTreeRegressor().fit(X_train, y_train)
y_preds = dtr.predict(X_test)
mean_squared_error(y_test, y_preds)

76.70510382118549

In [15]:
rfr = RandomForestRegressor().fit(X_train, y_train)
y_preds = rfr.predict(X_test)
mean_squared_error(y_test, y_preds)

In [ ]:
X_forecast = df_test_actual[df_test_actual.columns[:-1]].values
y_forecast_pred = rfr.predict(X_forecast)
y_forecast_pred = list(map(round, y_forecast_pred))

submission = pd.DataFrame()
submission['ID'] = df_test['ID']
submission['item_cnt_month'] = y_forecast_pred # predictions on the test dataset

In [ ]:
submission.to_csv('rfr_notune.csv', index=False)

### New Preprocessing

In [16]:
# make a ton of dataframes for all files
df_items = pd.read_csv('items.csv')
df_cat = pd.read_csv('item_categories.csv')
df_sales = pd.read_csv('sales_train.csv')
df_test = pd.read_csv('test.csv')
df_shops = pd.read_csv('shops.csv')
sample = pd.read_csv('sample_submission.csv')

In [17]:
# fix shops that are the same thing
df_shops.loc[df_shops['shop_id']==0, 'shop_id'] = 57
df_shops.loc[df_shops['shop_id']==1, 'shop_id'] = 58
df_shops.loc[df_shops['shop_id']==10, 'shop_id'] = 11
# df_shops

In [18]:
# outlier removal
df_sales = df_sales[df_sales['item_price'] > 0]
df_sales = df_sales[df_sales['item_price'] < 50000]
df_sales = df_sales[df_sales['item_cnt_day'] > 0]
df_sales = df_sales[df_sales['item_cnt_day'] < 1000]

In [19]:
# fix shop ID in the main dfs
df_sales.loc[df_sales['shop_id'] == 0, 'shop_id'] = 57
df_sales.loc[df_sales['shop_id'] == 1, 'shop_id'] = 58
df_sales.loc[df_sales['shop_id'] == 10, 'shop_id'] = 11
df_sales.loc[df_sales['shop_id'] == 39, 'shop_id'] = 40

df_test.loc[df_test['shop_id'] == 0, 'shop_id'] = 57
df_test.loc[df_test['shop_id'] == 1, 'shop_id'] = 58
df_test.loc[df_test['shop_id'] == 10, 'shop_id'] = 11
df_test.loc[df_test['shop_id'] == 39, 'shop_id'] = 40

In [16]:
df_items

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40
...,...,...,...
22165,"Ядерный титбит 2 [PC, Цифровая версия]",22165,31
22166,Язык запросов 1С:Предприятия [Цифровая версия],22166,54
22167,Язык запросов 1С:Предприятия 8 (+CD). Хрустале...,22167,49
22168,Яйцо для Little Inu,22168,62


### Feature Engineering - In all CSVs, create new cols

In [20]:
# df_shops

# create type and type_code cols
df_cat['type'] = df_cat['item_category_name'].str.split('-').map(lambda x:x[0])
df_cat['type_code'] = LabelEncoder().fit_transform(df_cat['type'])

# get subtype and subtype_code cols
df_cat['sub_type'] = df_cat['item_category_name'].str.split('-').map(lambda x:x[1].strip() if len(x) > 1 else x[0].strip())
df_cat['sub_type_code'] = LabelEncoder().fit_transform(df_cat['sub_type'])
df_cat.head()

,item_category_name,item_category_id,type,type_code,sub_type,sub_type_code
0,PC - Гарнитуры/Наушники,0,PC,0,Гарнитуры/Наушники,29
1,Аксессуары - PS2,1,Аксессуары,1,PS2,9
2,Аксессуары - PS3,2,Аксессуары,1,PS3,10
3,Аксессуары - PS4,3,Аксессуары,1,PS4,11
4,Аксессуары - PSP,4,Аксессуары,1,PSP,13


In [21]:
# df_sales

# replace the negative item_price to mean value
mean = df_sales[(df_sales["shop_id"] == 32) & (df_sales["item_id"] == 2973) & 
                (df_sales["date_block_num"] == 4) & (df_sales["item_price"] > 0)]["item_price"].mean()
df_sales.loc[df_sales.item_price < 0, 'item_price'] = mean

# clean the outliers
df_sales = df_sales[df_sales["item_price"] < np.percentile(df_sales["item_price"], q = 100)]
df_sales = df_sales[(df_sales["item_cnt_day"] >= 0) & (df_sales["item_cnt_day"] < np.percentile(df_sales["item_cnt_day"], q = 100))]

# change the format of date
df_sales["date"] = pd.to_datetime(df_sales["date"], format = "%d.%m.%Y")
df_sales.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,2013-01-02,0,59,22154,999.00,1.0
1,2013-01-03,0,25,2552,899.00,1.0
3,2013-01-06,0,25,2554,1709.05,1.0
4,2013-01-15,0,25,2555,1099.00,1.0
5,2013-01-10,0,25,2564,349.00,1.0


In [22]:
for i in [(0, 57), (1, 58), (10, 11)]:
    df_sales.loc[df_sales['shop_id'] == i[0], 'shop_id'] = i[1]
    df_test.loc[df_test['shop_id'] == i[0], 'shop_id'] = i[1]

In [23]:
df_sales['revenue'] = df_sales['item_cnt_day'] * df_sales['item_price']

In [24]:
# df_items

# split item names by first bracket
df_items['name1'], df_items['name2'] = df_items['item_name'].str.split('[', 1).str
df_items['name1'], df_items['name3'] = df_items['item_name'].str.split('(', 1).str

# replace special characters and turn to lower case
df_items['name2'] = df_items['name2'].str.replace('[^A-Za-z0-9А-Яа-я]+', " ").str.lower()
df_items['name3'] = df_items['name3'].str.replace('[^A-Za-z0-9А-Яа-я]+', " ").str.lower()

# fill nulls with '0'
df_items = df_items.fillna('0')

# return all characters except the last if name 2 is not "0" - the closing bracket
df_items['name2'] = df_items['name2'].apply(lambda x: x[:-1] if x != '0' else '0')

<ipython-input-24-f6abc034c128>:4: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  df_items['name1'], df_items['name2'] = df_items['item_name'].str.split('[', 1).str
<ipython-input-24-f6abc034c128>:5: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  df_items['name1'], df_items['name3'] = df_items['item_name'].str.split('(', 1).str


In [25]:
df_items['type'] = (df_items['name2'].apply(lambda x: x[0:8] if x.split(' ')[0] == 'xbox' else x.split(' ')[0]))

df_items.loc[(df_items['type'] == 'x360')|(df_items['type'] == 'xbox360')|(df_items['type'] == 'xbox 360'),'type'] = 'xbox 360'
df_items.loc[df_items['type'] == '', 'type'] = 'mac'
df_items.type = (df_items['type'].apply(lambda x: x.replace(' ', '')))
df_items.loc[(df_items['type'] == 'pc' )|(df_items['type'] == 'pс')|(df_items['type'] == 'pс'),'type'] = 'pс'

df_items.loc[df_items['type'] == 'рs3' , 'type'] = 'рs3'

In [26]:
# Create "grid" with columns
index_cols = ['shop_id', 'item_id', 'date_block_num']

# For every month we create a grid from all shops/items combinations from that month
grid = [] 
for month in df_sales['date_block_num'].unique():
    cur_shops = df_sales.loc[df_sales['date_block_num'] == month, 'shop_id'].unique()
    cur_items = df_sales.loc[df_sales['date_block_num'] == month, 'item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [month]])),dtype='int32'))

# Turn the grid into a dataframe
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

# Groupby data to get shop-item-month aggregates
gb = df_sales.groupby(index_cols,as_index=False).agg({'item_cnt_day':'sum'})
gb.rename(columns ={'item_cnt_day':'item_cnt_month'},inplace = True)
train_data = pd.merge(grid, gb, how='left', on=index_cols).fillna(0)

#Clip target values
train_data['item_cnt_month'] = np.clip(train_data['item_cnt_month'],0,20)
train_data.sort_values(index_cols, inplace = True)

In [27]:
df_test.insert(loc=3, column='date_block_num', value=34)
train_data = train_data.append(df_test.drop('ID', axis = 1)).fillna(0)
train_data.head().append(train_data.tail())

,shop_id,item_id,date_block_num,item_cnt_month
7118778,2,0,20,0.0
5646744,2,1,15,0.0
6711465,2,1,18,0.0
7026005,2,1,19,0.0
7121093,2,1,20,0.0
214195,45,18454,34,0.0
214196,45,16188,34,0.0
214197,45,15757,34,0.0
214198,45,19648,34,0.0
214199,45,969,34,0.0


### Modeling again - XGB, DTR, RFR

In [17]:
X_train = train_data[train_data.date_block_num < 33].drop(['item_cnt_month'], axis=1)
Y_train = train_data[train_data.date_block_num < 33]['item_cnt_month']
X_valid = train_data[train_data.date_block_num == 33].drop(['item_cnt_month'], axis=1)
Y_valid = train_data[train_data.date_block_num == 33]['item_cnt_month']
X_test = train_data[train_data.date_block_num == 34].drop(['item_cnt_month'], axis=1)

In [104]:
model = XGBRegressor(
    max_depth=8,
    n_estimators=1000,
    min_child_weight=300, 
    colsample_bytree=0.8, 
    subsample=0.8, 
    eta=0.1,    
    seed=42)

model.fit(
    X_train,
    Y_train,
    eval_metric="rmse",
    eval_set=[(X_train, Y_train), (X_valid, Y_valid)],
    verbose=True,
    early_stopping_rounds = 10)

[0]	validation_0-rmse:1.24241	validation_1-rmse:1.15543
[1]	validation_0-rmse:1.23508	validation_1-rmse:1.14944
[2]	validation_0-rmse:1.23072	validation_1-rmse:1.14457
[3]	validation_0-rmse:1.22635	validation_1-rmse:1.14031
[4]	validation_0-rmse:1.22175	validation_1-rmse:1.13694
[5]	validation_0-rmse:1.22020	validation_1-rmse:1.13406
[6]	validation_0-rmse:1.21869	validation_1-rmse:1.13183
[7]	validation_0-rmse:1.21673	validation_1-rmse:1.12993
[8]	validation_0-rmse:1.21523	validation_1-rmse:1.12831
[9]	validation_0-rmse:1.21384	validation_1-rmse:1.12701
[10]	validation_0-rmse:1.21218	validation_1-rmse:1.12596
[11]	validation_0-rmse:1.21044	validation_1-rmse:1.12506
[12]	validation_0-rmse:1.20948	validation_1-rmse:1.12424
[13]	validation_0-rmse:1.20877	validation_1-rmse:1.12354
[14]	validation_0-rmse:1.20775	validation_1-rmse:1.12309
[15]	validation_0-rmse:1.20663	validation_1-rmse:1.12266
[16]	validation_0-rmse:1.20619	validation_1-rmse:1.12223
[17]	validation_0-rmse:1.20571	validation

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, eta=0.1, gamma=0,
             gpu_id=-1, importance_type='gain', interaction_constraints='',
             learning_rate=0.100000001, max_delta_step=0, max_depth=8,
             min_child_weight=300, missing=nan, monotone_constraints='()',
             n_estimators=1000, n_jobs=12, num_parallel_tree=1, random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=42,
             subsample=0.8, tree_method='approx', validate_parameters=1,
             verbosity=None)

In [105]:
y_pred = model.predict(X_test).clip(0,20)
sample['item_cnt_month'] = y_pred 
sample.to_csv('xgb_try1.csv', index=False)

D:\Anaconda\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


In [106]:
dtr = DecisionTreeRegressor().fit(X_train, Y_train)
y_preds = dtr.predict(X_valid)
mean_squared_error(Y_valid, y_preds)

1.6384847364795618

In [107]:
y_pred = dtr.predict(X_test).clip(0,20)
sample['item_cnt_month'] = y_pred
sample.to_csv('dtr_try2.csv', index=False)

In [109]:
rfr = RandomForestRegressor().fit(X_train, Y_train)
y_preds = rfr.predict(X_valid)
mean_squared_error(Y_valid, y_preds)

In [ ]:
y_pred = rfr.predict(X_test).clip(0,20)
sample['item_cnt_month'] = y_pred
sample.to_csv('rfr_try2.csv', index=False)

In [110]:
train_data

,shop_id,item_id,date_block_num,item_cnt_month
7118778,2,0,20,0.0
5646744,2,1,15,0.0
6711465,2,1,18,0.0
7026005,2,1,19,0.0
7121093,2,1,20,0.0
...,...,...,...,...
214195,45,18454,34,0.0
214196,45,16188,34,0.0
214197,45,15757,34,0.0
214198,45,19648,34,0.0


In [25]:
#lag feature
for i in [1,2,3,6,12]:
    shifted = train_data[['date_block_num','shop_id','item_id','item_cnt_month']].copy()
    shifted.columns = ['date_block_num','shop_id','item_id', 'item_cnt_month'+'_lag_'+str(i)]
    shifted['date_block_num'] += i
    train_data = pd.merge(train_data, shifted, on=['date_block_num','shop_id','item_id'], how='left')

In [26]:
group = train_data.groupby(['date_block_num','item_id'])['item_cnt_month'].mean().rename('date_item_avg_cnt').reset_index()
train_data = pd.merge(train_data, group, on=['date_block_num', 'item_id'], how='left')
for i in [1,2,3,6,12]:
    shifted = train_data[['date_block_num','shop_id','item_id','date_item_avg_cnt']].copy()
    shifted.columns = ['date_block_num','shop_id','item_id', 'date_item_avg_cnt'+'_lag_'+str(i)]
    shifted['date_block_num'] += i
    train_data = pd.merge(train_data, shifted, on=['date_block_num','shop_id','item_id'], how='left')
train_data.drop(['date_item_avg_cnt'], axis=1, inplace=True)

In [27]:
group = train_data.groupby(['date_block_num','shop_id'])['item_cnt_month'].mean().rename('date_shop_avg_cnt').reset_index()
train_data = pd.merge(train_data, group, on=['date_block_num', 'shop_id'], how='left')
for i in [1,2,3,6,12]:
    shifted = train_data[['date_block_num','shop_id','item_id','date_shop_avg_cnt']].copy()
    shifted.columns = ['date_block_num','shop_id','item_id', 'date_shop_avg_cnt'+'_lag_'+str(i)]
    shifted['date_block_num'] += i
    train_data = pd.merge(train_data, shifted, on=['date_block_num','shop_id','item_id'], how='left')
train_data.drop(['date_shop_avg_cnt'], axis=1, inplace=True)

In [28]:
group = train_data.groupby(['date_block_num'])['item_cnt_month'].mean().rename('date_avg_cnt').reset_index()
train_data = pd.merge(train_data, group, on=['date_block_num'], how='left')
shifted = train_data[['date_block_num','shop_id','item_id','date_avg_cnt']].copy()
shifted.columns = ['date_block_num','shop_id','item_id', 'date_avg_cnt'+'_lag_'+str(1)]
shifted['date_block_num'] += 1
train_data = pd.merge(train_data, shifted, on=['date_block_num','shop_id','item_id'], how='left')
train_data.drop(['date_avg_cnt'], axis=1, inplace=True)

In [29]:
group = train_data.groupby(['date_block_num','shop_id','item_id'])['item_cnt_month'].mean().rename('date_shop_item_avg_cnt').reset_index()
train_data = pd.merge(train_data, group, on=['date_block_num', 'shop_id','item_id'], how='left')
for i in [1,2,3]:
    shifted = train_data[['date_block_num','shop_id','item_id','date_shop_item_avg_cnt']].copy()
    shifted.columns = ['date_block_num','shop_id','item_id', 'date_shop_item_avg_cnt'+'_lag_'+str(i)]
    shifted['date_block_num'] += i
    train_data = pd.merge(train_data, shifted, on=['date_block_num','shop_id','item_id'], how='left')
train_data.drop(['date_shop_item_avg_cnt'], axis=1, inplace=True)

In [30]:
#trend
group = df_sales.groupby(['item_id'])['item_price'].mean().rename('item_avg_price').reset_index()
train_data = pd.merge(train_data, group, on=['item_id'], how='left')

In [31]:
group = df_sales.groupby(['date_block_num','item_id'])['item_price'].mean().rename('date_item_avg_price').reset_index()
train_data = pd.merge(train_data, group, on=['date_block_num','item_id'], how='left')

In [32]:
lags = [1,2,3,4,5,6]
for i in lags:
    shifted = train_data[['date_block_num','shop_id','item_id','date_item_avg_price']].copy()
    shifted.columns = ['date_block_num','shop_id','item_id', 'date_item_avg_price'+'_lag_'+str(i)]
    shifted['date_block_num'] += i
    train_data = pd.merge(train_data, shifted, on=['date_block_num','shop_id','item_id'], how='left')

In [33]:
for i in lags:
    train_data['trend_price_lag_'+str(i)] = (train_data['date_item_avg_price_lag_'+str(i)] - train_data['item_avg_price']) / train_data['item_avg_price']

In [34]:
def trend(row):
    for i in lags:
        if row['trend_price_lag_'+str(i)]:
            return row['trend_price_lag_'+str(i)]
    return 0
train_data['trend_price_lag'] = train_data.apply(trend, axis=1)

In [35]:
fetures_to_drop = ['item_avg_price', 'date_item_avg_price']
for i in lags:
    fetures_to_drop += ['date_item_avg_price_lag_'+str(i)]
    fetures_to_drop += ['trend_price_lag_'+str(i)]
train_data.drop(fetures_to_drop, axis=1, inplace=True)

In [36]:
group = df_sales.groupby(['date_block_num','shop_id',])['revenue'].sum().rename('sum_date_shops_revenue').reset_index()
train_data = pd.merge(train_data, group, on=['date_block_num','shop_id'], how='left')

In [37]:
group = group.groupby(['shop_id',])['sum_date_shops_revenue'].mean().rename('mean_shops_revenue').reset_index()
train_data = pd.merge(train_data, group, on=['shop_id'], how='left')

In [38]:
train_data['trend_revenue'] = (train_data['sum_date_shops_revenue'] - train_data['mean_shops_revenue']) / train_data['mean_shops_revenue']
shifted = train_data[['date_block_num','shop_id','item_id','trend_revenue']].copy()
shifted.columns = ['date_block_num','shop_id','item_id', 'trend_revenue'+'_lag_'+str(1)]
shifted['date_block_num'] += 1
train_data = pd.merge(train_data, shifted, on=['date_block_num','shop_id','item_id'], how='left')

In [39]:
train_data.drop(['sum_date_shops_revenue'], axis=1, inplace=True)
train_data.drop(['mean_shops_revenue'], axis=1, inplace=True)
train_data.drop(['trend_revenue'], axis=1, inplace=True)

In [40]:
train_data['item_shop_first_sale'] = (
    train_data['date_block_num'] - train_data.groupby(['item_id', 'shop_id'])['date_block_num'].transform('min')
)
train_data['item_first_sale'] = (
    train_data['date_block_num'] - train_data.groupby(['item_id'])['date_block_num'].transform('min')
)

In [41]:
train_data.fillna(0,inplace = True)

In [42]:
train_data = train_data[train_data['date_block_num'] > 3]
train_data.head().append(train_data.tail())

,shop_id,item_id,date_block_num,item_cnt_month,item_cnt_month_lag_1,item_cnt_month_lag_2,item_cnt_month_lag_3,item_cnt_month_lag_6,item_cnt_month_lag_12,date_item_avg_cnt_lag_1,...,date_shop_avg_cnt_lag_6,date_shop_avg_cnt_lag_12,date_avg_cnt_lag_1,date_shop_item_avg_cnt_lag_1,date_shop_item_avg_cnt_lag_2,date_shop_item_avg_cnt_lag_3,trend_price_lag,trend_revenue_lag_1,item_shop_first_sale,item_first_sale
0,2,0,20,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0,0
1,2,1,15,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0,0
2,2,1,18,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0000,0.000000,0.0,0.0,0.0,0.000000,0.000000,3,3
3,2,1,19,0.0,0.0,0.0,0.0,0.0,0.0,0.020408,...,0.000000,0.0000,0.260610,0.0,0.0,0.0,0.000000,-0.148407,4,4
4,2,1,20,0.0,0.0,0.0,0.0,0.0,0.0,0.020000,...,0.000000,0.0000,0.296927,0.0,0.0,0.0,0.000000,-0.081304,5,5
11026914,45,18454,34,0.0,1.0,0.0,0.0,0.0,0.0,0.045455,...,0.137611,0.0000,0.259152,1.0,0.0,0.0,-0.475305,-0.292428,11,11
11026915,45,16188,34,0.0,0.0,0.0,0.0,0.0,0.0,0.022727,...,0.000000,0.0000,0.259152,0.0,0.0,0.0,0.081167,-0.292428,2,2
11026916,45,15757,34,0.0,0.0,0.0,0.0,0.0,0.0,0.113636,...,0.137611,0.1813,0.259152,0.0,0.0,0.0,0.155919,-0.292428,34,34
11026917,45,19648,34,0.0,0.0,0.0,0.0,0.0,0.0,0.045455,...,0.137611,0.0000,0.259152,0.0,0.0,0.0,-0.091709,-0.292428,11,11
11026918,45,969,34,0.0,0.0,0.0,0.0,0.0,0.0,0.068182,...,0.137611,0.1813,0.259152,0.0,0.0,0.0,-0.605811,-0.292428,17,17


In [43]:
X_train = train_data[train_data.date_block_num < 33].drop(['item_cnt_month'], axis=1)
Y_train = train_data[train_data.date_block_num < 33]['item_cnt_month']
X_valid = train_data[train_data.date_block_num == 33].drop(['item_cnt_month'], axis=1)
Y_valid = train_data[train_data.date_block_num == 33]['item_cnt_month']
X_test = train_data[train_data.date_block_num == 34].drop(['item_cnt_month'], axis=1)

In [37]:
xgb_model = XGBRegressor()

In [38]:
param_grid = {
    'max_depth':[1,2,4],
    'n_estimators':[1, 10, 100],
    'min_child_weight':[100, 200, 300],
    'colsample_bytree':[0.2, 0.4, 0.6],
    'seed':[42]
}

In [39]:
cv = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv= 5)

In [40]:
cv.fit(
    X_train,
    Y_train,
    eval_metric="rmse",
    eval_set=[(X_train, Y_train), (X_valid, Y_valid)],
    verbose=True,
    early_stopping_rounds = 5
)

[0]	validation_0-rmse:1.20151	validation_1-rmse:1.13556


D:\Anaconda\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


[0]	validation_0-rmse:1.20348	validation_1-rmse:1.13523


D:\Anaconda\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


[0]	validation_0-rmse:1.20506	validation_1-rmse:1.13511


...

D:\Anaconda\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


[0]	validation_0-rmse:1.10976	validation_1-rmse:1.06909
[1]	validation_0-rmse:1.05235	validation_1-rmse:1.02604
[2]	validation_0-rmse:1.00678	validation_1-rmse:0.99581
[3]	validation_0-rmse:0.98225	validation_1-rmse:0.98097
[4]	validation_0-rmse:0.96530	validation_1-rmse:0.97162
[5]	validation_0-rmse:0.95632	validation_1-rmse:0.96779
[6]	validation_0-rmse:0.95047	validation_1-rmse:0.96434
[7]	validation_0-rmse:0.94574	validation_1-rmse:0.96381
[8]	validation_0-rmse:0.94304	validation_1-rmse:0.96202
[9]	validation_0-rmse:0.94088	validation_1-rmse:0.96061
[10]	validation_0-rmse:0.93976	validation_1-rmse:0.95996
[11]	validation_0-rmse:0.93876	validation_1-rmse:0.95786
[12]	validation_0-rmse:0.93767	validation_1-rmse:0.95747
[13]	validation_0-rmse:0.93607	validation_1-rmse:0.95696
[14]	validation_0-rmse:0.93444	validation_1-rmse:0.95568
[15]	validation_0-rmse:0.93331	validation_1-rmse:0.95522
[16]	validation_0-rmse:0.93268	validation_1-rmse:0.95508
[17]	validation_0-rmse:0.93190	validation

D:\Anaconda\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


[0]	validation_0-rmse:1.10887	validation_1-rmse:1.06880
[1]	validation_0-rmse:1.05130	validation_1-rmse:1.02583
[2]	validation_0-rmse:1.00534	validation_1-rmse:0.99485
[3]	validation_0-rmse:0.98143	validation_1-rmse:0.98049
[4]	validation_0-rmse:0.96404	validation_1-rmse:0.97213
[5]	validation_0-rmse:0.95547	validation_1-rmse:0.96743
[6]	validation_0-rmse:0.94939	validation_1-rmse:0.96402
[7]	validation_0-rmse:0.94534	validation_1-rmse:0.96358
[8]	validation_0-rmse:0.94245	validation_1-rmse:0.96154
[9]	validation_0-rmse:0.94056	validation_1-rmse:0.96058
[10]	validation_0-rmse:0.93918	validation_1-rmse:0.95930
[11]	validation_0-rmse:0.93838	validation_1-rmse:0.95871
[12]	validation_0-rmse:0.93696	validation_1-rmse:0.95843
[13]	validation_0-rmse:0.93534	validation_1-rmse:0.95792
[14]	validation_0-rmse:0.93385	validation_1-rmse:0.95680
[15]	validation_0-rmse:0.93312	validation_1-rmse:0.95621
[16]	validation_0-rmse:0.93193	validation_1-rmse:0.95528
[17]	validation_0-rmse:0.93127	validation

GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, gamma=None,
                                    gpu_id=None, importance_type='gain',
                                    interaction_constraints=None,
                                    learning_rate=None, max_delta_step=None,
                                    max_depth=None, min_child_weight=None,
                                    missing=nan, monotone_constraints=None,
                                    n_estimators=100, n_jobs=None,
                                    num_parallel_tree=None, random_state=None,
                                    reg_alpha=None, reg_lambda=None,
                                    scale_pos_weight=None, subsample=None,
                                    tree_method=None, validate_para

In [41]:
y_pred = cv.predict(X_test).clip(0,20)
sample['item_cnt_month'] = y_pred 
sample.to_csv('xgb_cvfeat.csv', index=False)

D:\Anaconda\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


In [43]:
model = XGBRegressor(
    max_depth=8,
    n_estimators=1000,
    min_child_weight=300, 
    colsample_bytree=0.8, 
    subsample=0.8, 
    eta=0.1,    
    seed=42)

model.fit(
    X_train,
    Y_train,
    eval_metric="rmse",
    eval_set=[(X_train, Y_train), (X_valid, Y_valid)],
    verbose=True,
    early_stopping_rounds = 10)

[0]	validation_0-rmse:1.19034	validation_1-rmse:1.12468
[1]	validation_0-rmse:1.14488	validation_1-rmse:1.09202
[2]	validation_0-rmse:1.10907	validation_1-rmse:1.06573
[3]	validation_0-rmse:1.07900	validation_1-rmse:1.04382
[4]	validation_0-rmse:1.05478	validation_1-rmse:1.02493
[5]	validation_0-rmse:1.03270	validation_1-rmse:1.01059
[6]	validation_0-rmse:1.01392	validation_1-rmse:0.99864
[7]	validation_0-rmse:0.99862	validation_1-rmse:0.98865
[8]	validation_0-rmse:0.98582	validation_1-rmse:0.98061
[9]	validation_0-rmse:0.97474	validation_1-rmse:0.97371
[10]	validation_0-rmse:0.96615	validation_1-rmse:0.96817
[11]	validation_0-rmse:0.95659	validation_1-rmse:0.96333
[12]	validation_0-rmse:0.94973	validation_1-rmse:0.96002
[13]	validation_0-rmse:0.94456	validation_1-rmse:0.95707
[14]	validation_0-rmse:0.93990	validation_1-rmse:0.95444
[15]	validation_0-rmse:0.93534	validation_1-rmse:0.95242
[16]	validation_0-rmse:0.93180	validation_1-rmse:0.95091
[17]	validation_0-rmse:0.92871	validation

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, eta=0.1, gamma=0,
             gpu_id=-1, importance_type='gain', interaction_constraints='',
             learning_rate=0.100000001, max_delta_step=0, max_depth=8,
             min_child_weight=300, missing=nan, monotone_constraints='()',
             n_estimators=1000, n_jobs=12, num_parallel_tree=1, random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=42,
             subsample=0.8, tree_method='approx', validate_parameters=1,
             verbosity=None)

In [45]:
y_pred = model.predict(X_test).clip(0,20)
sample['item_cnt_month'] = y_pred 
sample.to_csv('xgb_feat.csv', index=False)

In [44]:
dtr = DecisionTreeRegressor().fit(X_train, Y_train)
y_preds = dtr.predict(X_valid)
mean_squared_error(Y_valid, y_preds)

1.932595218494313

In [151]:
y_pred = dtr.predict(X_test).clip(0,20)
sample['item_cnt_month'] = y_pred
sample.to_csv('dtr_lag.csv', index=False)

In [ ]:
rfr = RandomForestRegressor().fit(X_train, Y_train)
y_preds = rfr.predict(X_valid)
mean_squared_error(Y_valid, y_preds)

In [153]:
y_pred = rfr.predict(X_test).clip(0,20)
sample['item_cnt_month'] = y_pred
sample.to_csv('rfr_lag.csv', index=False)

In [ ]:
df_sales = df_sales[df_sales['item_price'] > 0]
df_sales = df_sales[df_sales['item_price'] < 50000]
df_sales = df_sales[df_sales['item_cnt_day'] > 0]
df_sales = df_sales[df_sales['item_cnt_day'] < 1000]

In [53]:
df_sales.loc[df_sales:]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,revenue
2720000,2015-07-23,30,4,5820,3199.0,1.0,3199.0
2720001,2015-07-22,30,4,5816,1199.0,1.0,1199.0
2720002,2015-07-05,30,4,5814,1199.0,1.0,1199.0
2720003,2015-07-23,30,4,5516,299.0,1.0,299.0
2720004,2015-07-27,30,4,5490,1399.0,1.0,1399.0
...,...,...,...,...,...,...,...
2935844,2015-10-10,33,25,7409,299.0,1.0,299.0
2935845,2015-10-09,33,25,7460,299.0,1.0,299.0
2935846,2015-10-14,33,25,7459,349.0,1.0,349.0
2935847,2015-10-22,33,25,7440,299.0,1.0,299.0


In [56]:
# df_shops

# create type and type_code cols
df_cat['type'] = df_cat['item_category_name'].str.split('-').map(lambda x:x[0])
df_cat['type_code'] = LabelEncoder().fit_transform(df_cat['type'])

# get subtype and subtype_code cols
df_cat['sub_type'] = df_cat['item_category_name'].str.split('-').map(lambda x:x[1].strip() if len(x) > 1 else x[0].strip())
df_cat['sub_type_code'] = LabelEncoder().fit_transform(df_cat['sub_type'])
df_cat.head()

,item_category_name,item_category_id,type,type_code,sub_type,sub_type_code
0,PC - Гарнитуры/Наушники,0,PC,0,Гарнитуры/Наушники,29
1,Аксессуары - PS2,1,Аксессуары,1,PS2,9
2,Аксессуары - PS3,2,Аксессуары,1,PS3,10
3,Аксессуары - PS4,3,Аксессуары,1,PS4,11
4,Аксессуары - PSP,4,Аксессуары,1,PSP,13


In [57]:
# df_sales

# replace the negative item_price to mean value
mean = df_sales[(df_sales["shop_id"] == 32) & (df_sales["item_id"] == 2973) & 
                (df_sales["date_block_num"] == 4) & (df_sales["item_price"] > 0)]["item_price"].mean()
df_sales.loc[df_sales.item_price < 0, 'item_price'] = mean

# clean the outliers
df_sales = df_sales[df_sales["item_price"] < np.percentile(df_sales["item_price"], q = 100)]
df_sales = df_sales[(df_sales["item_cnt_day"] >= 0) & (df_sales["item_cnt_day"] < np.percentile(df_sales["item_cnt_day"], q = 100))]

# change the format of date
df_sales["date"] = pd.to_datetime(df_sales["date"], format = "%d.%m.%Y")
df_sales.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,revenue
0,2013-01-02,0,59,22154,999.00,1.0,999.00
1,2013-01-03,0,25,2552,899.00,1.0,899.00
3,2013-01-06,0,25,2554,1709.05,1.0,1709.05
4,2013-01-15,0,25,2555,1099.00,1.0,1099.00
5,2013-01-10,0,25,2564,349.00,1.0,349.00


In [58]:
for i in [(0, 57), (1, 58), (10, 11)]:
    df_sales.loc[df_sales['shop_id'] == i[0], 'shop_id'] = i[1]
    df_test.loc[df_test['shop_id'] == i[0], 'shop_id'] = i[1]

In [59]:
df_sales['revenue'] = df_sales['item_cnt_day'] * df_sales['item_price']

In [60]:
# df_items

# split item names by first bracket
df_items['name1'], df_items['name2'] = df_items['item_name'].str.split('[', 1).str
df_items['name1'], df_items['name3'] = df_items['item_name'].str.split('(', 1).str

# replace special characters and turn to lower case
df_items['name2'] = df_items['name2'].str.replace('[^A-Za-z0-9А-Яа-я]+', " ").str.lower()
df_items['name3'] = df_items['name3'].str.replace('[^A-Za-z0-9А-Яа-я]+', " ").str.lower()

# fill nulls with '0'
df_items = df_items.fillna('0')

# return all characters except the last if name 2 is not "0" - the closing bracket
df_items['name2'] = df_items['name2'].apply(lambda x: x[:-1] if x != '0' else '0')

<ipython-input-60-f6abc034c128>:4: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  df_items['name1'], df_items['name2'] = df_items['item_name'].str.split('[', 1).str
<ipython-input-60-f6abc034c128>:5: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  df_items['name1'], df_items['name3'] = df_items['item_name'].str.split('(', 1).str


In [61]:
df_items['type'] = (df_items['name2'].apply(lambda x: x[0:8] if x.split(' ')[0] == 'xbox' else x.split(' ')[0]))

df_items.loc[(df_items['type'] == 'x360')|(df_items['type'] == 'xbox360')|(df_items['type'] == 'xbox 360'),'type'] = 'xbox 360'
df_items.loc[df_items['type'] == '', 'type'] = 'mac'
df_items.type = (df_items['type'].apply(lambda x: x.replace(' ', '')))
df_items.loc[(df_items['type'] == 'pc' )|(df_items['type'] == 'pс')|(df_items['type'] == 'pс'),'type'] = 'pс'

df_items.loc[df_items['type'] == 'рs3' , 'type'] = 'рs3'

In [62]:
# Create "grid" with columns
index_cols = ['shop_id', 'item_id', 'date_block_num']

# For every month we create a grid from all shops/items combinations from that month
grid = [] 
for month in df_sales['date_block_num'].unique():
    cur_shops = df_sales.loc[df_sales['date_block_num'] == month, 'shop_id'].unique()
    cur_items = df_sales.loc[df_sales['date_block_num'] == month, 'item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [month]])),dtype='int32'))

# Turn the grid into a dataframe
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

# Groupby data to get shop-item-month aggregates
gb = df_sales.groupby(index_cols,as_index=False).agg({'item_cnt_day':'sum'})
gb.rename(columns ={'item_cnt_day':'item_cnt_month'},inplace = True)
train_data = pd.merge(grid, gb, how='left', on=index_cols).fillna(0)

#Clip target values
train_data['item_cnt_month'] = np.clip(train_data['item_cnt_month'],0,20)
train_data.sort_values(index_cols, inplace = True)

In [64]:
#lag feature
for i in [1,2,3,6,12]:
    shifted = train_data[['date_block_num','shop_id','item_id','item_cnt_month']].copy()
    shifted.columns = ['date_block_num','shop_id','item_id', 'item_cnt_month'+'_lag_'+str(i)]
    shifted['date_block_num'] += i
    train_data = pd.merge(train_data, shifted, on=['date_block_num','shop_id','item_id'], how='left')

In [65]:
group = train_data.groupby(['date_block_num','item_id'])['item_cnt_month'].mean().rename('date_item_avg_cnt').reset_index()
train_data = pd.merge(train_data, group, on=['date_block_num', 'item_id'], how='left')
for i in [1,2,3,6,12]:
    shifted = train_data[['date_block_num','shop_id','item_id','date_item_avg_cnt']].copy()
    shifted.columns = ['date_block_num','shop_id','item_id', 'date_item_avg_cnt'+'_lag_'+str(i)]
    shifted['date_block_num'] += i
    train_data = pd.merge(train_data, shifted, on=['date_block_num','shop_id','item_id'], how='left')
train_data.drop(['date_item_avg_cnt'], axis=1, inplace=True)

In [66]:
group = train_data.groupby(['date_block_num','shop_id'])['item_cnt_month'].mean().rename('date_shop_avg_cnt').reset_index()
train_data = pd.merge(train_data, group, on=['date_block_num', 'shop_id'], how='left')
for i in [1,2,3,6,12]:
    shifted = train_data[['date_block_num','shop_id','item_id','date_shop_avg_cnt']].copy()
    shifted.columns = ['date_block_num','shop_id','item_id', 'date_shop_avg_cnt'+'_lag_'+str(i)]
    shifted['date_block_num'] += i
    train_data = pd.merge(train_data, shifted, on=['date_block_num','shop_id','item_id'], how='left')
train_data.drop(['date_shop_avg_cnt'], axis=1, inplace=True)

In [67]:
group = train_data.groupby(['date_block_num'])['item_cnt_month'].mean().rename('date_avg_cnt').reset_index()
train_data = pd.merge(train_data, group, on=['date_block_num'], how='left')
shifted = train_data[['date_block_num','shop_id','item_id','date_avg_cnt']].copy()
shifted.columns = ['date_block_num','shop_id','item_id', 'date_avg_cnt'+'_lag_'+str(1)]
shifted['date_block_num'] += 1
train_data = pd.merge(train_data, shifted, on=['date_block_num','shop_id','item_id'], how='left')
train_data.drop(['date_avg_cnt'], axis=1, inplace=True)

In [68]:
group = train_data.groupby(['date_block_num','shop_id','item_id'])['item_cnt_month'].mean().rename('date_shop_item_avg_cnt').reset_index()
train_data = pd.merge(train_data, group, on=['date_block_num', 'shop_id','item_id'], how='left')
for i in [1,2,3]:
    shifted = train_data[['date_block_num','shop_id','item_id','date_shop_item_avg_cnt']].copy()
    shifted.columns = ['date_block_num','shop_id','item_id', 'date_shop_item_avg_cnt'+'_lag_'+str(i)]
    shifted['date_block_num'] += i
    train_data = pd.merge(train_data, shifted, on=['date_block_num','shop_id','item_id'], how='left')
train_data.drop(['date_shop_item_avg_cnt'], axis=1, inplace=True)

In [69]:
#trend
group = df_sales.groupby(['item_id'])['item_price'].mean().rename('item_avg_price').reset_index()
train_data = pd.merge(train_data, group, on=['item_id'], how='left')

In [70]:
group = df_sales.groupby(['date_block_num','item_id'])['item_price'].mean().rename('date_item_avg_price').reset_index()
train_data = pd.merge(train_data, group, on=['date_block_num','item_id'], how='left')

In [71]:
lags = [1,2,3,4,5,6]
for i in lags:
    shifted = train_data[['date_block_num','shop_id','item_id','date_item_avg_price']].copy()
    shifted.columns = ['date_block_num','shop_id','item_id', 'date_item_avg_price'+'_lag_'+str(i)]
    shifted['date_block_num'] += i
    train_data = pd.merge(train_data, shifted, on=['date_block_num','shop_id','item_id'], how='left')

In [72]:
for i in lags:
    train_data['trend_price_lag_'+str(i)] = (train_data['date_item_avg_price_lag_'+str(i)] - train_data['item_avg_price']) / train_data['item_avg_price']

In [73]:
def trend(row):
    for i in lags:
        if row['trend_price_lag_'+str(i)]:
            return row['trend_price_lag_'+str(i)]
    return 0
train_data['trend_price_lag'] = train_data.apply(trend, axis=1)

In [74]:
fetures_to_drop = ['item_avg_price', 'date_item_avg_price']
for i in lags:
    fetures_to_drop += ['date_item_avg_price_lag_'+str(i)]
    fetures_to_drop += ['trend_price_lag_'+str(i)]
train_data.drop(fetures_to_drop, axis=1, inplace=True)

In [75]:
group = df_sales.groupby(['date_block_num','shop_id',])['revenue'].sum().rename('sum_date_shops_revenue').reset_index()
train_data = pd.merge(train_data, group, on=['date_block_num','shop_id'], how='left')

In [76]:
group = df_sales.groupby(['date_block_num','shop_id',])['revenue'].sum().rename('sum_date_shops_revenue').reset_index()
train_data = pd.merge(train_data, group, on=['date_block_num','shop_id'], how='left')

In [77]:
group = group.groupby(['shop_id',])['sum_date_shops_revenue'].mean().rename('mean_shops_revenue').reset_index()
train_data = pd.merge(train_data, group, on=['shop_id'], how='left')

In [79]:
train_data['trend_revenue'] = (train_data['sum_date_shops_revenue'] - train_data['mean_shops_revenue']) / train_data['mean_shops_revenue']
shifted = train_data[['date_block_num','shop_id','item_id','trend_revenue']].copy()
shifted.columns = ['date_block_num','shop_id','item_id', 'trend_revenue'+'_lag_'+str(1)]
shifted['date_block_num'] += 1
train_data = pd.merge(train_data, shifted, on=['date_block_num','shop_id','item_id'], how='left')

In [82]:
train_data.drop(['sum_date_shops_revenue'], axis=1, inplace=True)
train_data.drop(['mean_shops_revenue'], axis=1, inplace=True)
train_data.drop(['trend_revenue'], axis=1, inplace=True)

In [83]:
train_data['item_shop_first_sale'] = (
    train_data['date_block_num'] - train_data.groupby(['item_id', 'shop_id'])['date_block_num'].transform('min')
)
train_data['item_first_sale'] = (
    train_data['date_block_num'] - train_data.groupby(['item_id'])['date_block_num'].transform('min')
)

In [84]:
train_data.fillna(0,inplace = True)

In [85]:
train_data = train_data[train_data['date_block_num'] > 3]
train_data.head().append(train_data.tail())

,shop_id,item_id,date_block_num,item_cnt_month,item_cnt_month_lag_1,item_cnt_month_lag_2,item_cnt_month_lag_3,item_cnt_month_lag_6,item_cnt_month_lag_12,date_item_avg_cnt_lag_1,...,date_avg_cnt_lag_1,date_shop_item_avg_cnt_lag_1,date_shop_item_avg_cnt_lag_2,date_shop_item_avg_cnt_lag_3,trend_price_lag,sum_date_shops_revenue_x,sum_date_shops_revenue_y,mean_shops_revenue,item_shop_first_sale,item_first_sale
0,2,0,20,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.000000,1479538.0,1479538.0,1.300397e+06,0,0
1,2,1,15,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.000000,1084938.0,1084938.0,1.300397e+06,0,0
2,2,1,18,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.000000,1107408.5,1107408.5,1.300397e+06,3,3
3,2,1,19,0.0,0.0,0.0,0.0,0.0,0.0,0.020408,...,0.260610,0.0,0.0,0.0,0.000000,1194669.0,1194669.0,1.300397e+06,4,4
4,2,1,20,0.0,0.0,0.0,0.0,0.0,0.0,0.020000,...,0.296927,0.0,0.0,0.0,0.000000,1479538.0,1479538.0,1.300397e+06,5,5
10812711,59,22167,31,0.0,0.0,0.0,0.0,0.0,0.0,0.790698,...,0.259977,0.0,0.0,0.0,0.004416,1001123.0,1001123.0,1.335571e+06,23,23
10812712,59,22167,32,0.0,0.0,0.0,0.0,0.0,0.0,0.690476,...,0.288634,0.0,0.0,0.0,0.004416,1100853.0,1100853.0,1.335571e+06,24,24
10812713,59,22167,33,0.0,0.0,0.0,0.0,0.0,0.0,0.500000,...,0.270697,0.0,0.0,0.0,0.004416,974885.0,974885.0,1.335571e+06,25,25
10812717,59,22168,8,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.000000,1662343.0,1662343.0,1.335571e+06,8,8
10812718,59,22169,14,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.000000,1506388.0,1506388.0,1.335571e+06,0,0


In [86]:
X_train = train_data[train_data.date_block_num < 33].drop(['item_cnt_month'], axis=1)
Y_train = train_data[train_data.date_block_num < 33]['item_cnt_month']
X_valid = train_data[train_data.date_block_num == 33].drop(['item_cnt_month'], axis=1)
Y_valid = train_data[train_data.date_block_num == 33]['item_cnt_month']
X_test = train_data[train_data.date_block_num == 34].drop(['item_cnt_month'], axis=1)

In [87]:
model = XGBRegressor(
    max_depth=8,
    n_estimators=1000,
    min_child_weight=300, 
    colsample_bytree=0.8, 
    subsample=0.8, 
    eta=0.1,    
    seed=42)

saved_model = model.fit(
    X_train,
    Y_train,
    eval_metric="rmse",
    eval_set=[(X_train, Y_train), (X_valid, Y_valid)],
    verbose=True,
    early_stopping_rounds = 10)

In [88]:
y_pred = model.predict(X_test).clip(0,20)
sample['item_cnt_month'] = y_pred 
sample.to_csv('xgb_mon3.csv', index=False)

In [6]:
dtr = DecisionTreeRegressor().fit(X_train, Y_train)
y_preds = dtr.predict(X_valid)
mean_squared_error(Y_valid, y_preds)

1.414263515118312


In [7]:
y_pred = dtr.predict(X_test).clip(0,20)
sample['item_cnt_month'] = y_pred
sample.to_csv('dtr_mon3.csv', index=False)

In [8]:
rfr = RandomForestRegressor().fit(X_train, Y_train)
y_preds = rfr.predict(X_valid)
mean_squared_error(Y_valid, y_preds)

1.847283291835324


In [9]:
y_pred = rfr.predict(X_test).clip(0,20)
sample['item_cnt_month'] = y_pred
sample.to_csv('rfr_mon3.csv', index=False)

In [10]:
import lightgbm as lgb
params = {'metric': 'rmse',
          'num_leaves': 255,
          'learning_rate': 0.005,
          'feature_fraction': 0.75,
          'bagging_fraction': 0.75,
          'bagging_freq': 5,
          'force_col_wise' : True,
          'random_state': 10}

In [11]:
cat_features = ['shop_id', 'month']

# lgb train and valid dataset
dtrain = lgb.Dataset(X_train, Y_train)
dvalid = lgb.Dataset(X_valid, Y_valid)
 
# Train LightGBM model
lgb_model = lgb.train(params=params,
                      train_set=dtrain,
                      num_boost_round=1500,
                      valid_sets=(dtrain, dvalid),
                      early_stopping_rounds=150,
                      categorical_feature=cat_features,
                      verbose_eval=100)

In [12]:
y_pred = lgb.predict(X_test).clip(0,20)
sample['item_cnt_month'] = y_pred
sample.to_csv('lgb_mon3.csv', index=False)